In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import gym  # Assuming you have a gym-like environment
from gym import spaces
import random
from collections import deque
import matplotlib.pyplot as plt
import io
import imageio.v2 as imageio
import torch.nn.functional as F
from matplotlib.patches import FancyArrow
from shapely.geometry import Point, LineString, box
import os
from datetime import datetime
from torch.distributions import Gumbel
from torch_geometric.nn import GATConv
import math
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
#GST and attention based model integration
class GAT(nn.Module):
    def __init__(self, in_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.gat_conv = GATConv(in_channels, out_channels, heads=heads, concat=True, dropout=0.6)
        self.fc_out = nn.Linear(out_channels * heads, out_channels)

    def forward(self, x, edge_index):
        x = self.gat_conv(x, edge_index)
        x = F.elu(x)
        x = self.fc_out(x)
        return x

class InteractionBasedAttentionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, num_layers):
        super(InteractionBasedAttentionModel, self).__init__()
        self.gat_layers = nn.ModuleList([
            GAT(in_channels=input_dim if i == 0 else hidden_dim, out_channels=hidden_dim, heads=num_heads)
            for i in range(num_layers)
        ])
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.gumbel_softmax = GumbelSoftmax()

    def forward(self, x, edge_index):
        for gat_layer in self.gat_layers:
            x = gat_layer(x, edge_index)
        x = self.fc_out(x)
        return self.gumbel_softmax(x)

    def predict(self, agent_positions, agent_velocities):
        x = torch.tensor(agent_positions + agent_velocities, dtype=torch.float32).to(device)
        edge_index = self.construct_edge_index(len(agent_positions) + len(agent_velocities))
        predicted_trajectories = self.forward(x, edge_index)
        return predicted_trajectories.detach().cpu().numpy()

    def construct_edge_index(self, num_agents):
        edge_index = []
        for i in range(num_agents):
            for j in range(num_agents):
                if i != j:
                    edge_index.append([i, j])
        return torch.tensor(edge_index, dtype=torch.long).t().contiguous().to(device)


class GumbelSoftmax(nn.Module):
    def __init__(self, temperature=1.0):
        super(GumbelSoftmax, self).__init__()
        self.temperature = temperature

    def sample_gumbel(self, shape, eps=1e-20):
        U = torch.rand(shape).to(device)
        return -torch.log(-torch.log(U + eps) + eps)

    def gumbel_softmax_sample(self, logits):
        y = logits + self.sample_gumbel(logits.size())
        return F.softmax(y / self.temperature, dim=-1)

    def forward(self, logits):
        return self.gumbel_softmax_sample(logits)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class EnhancedGumbelSocialTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, output_dim, max_len=100):
        super(EnhancedGumbelSocialTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, max_len)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.gumbel_softmax = GumbelSoftmax()

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = self.fc_out(x)
        return self.gumbel_softmax(x)

    def predict(self, agent_positions, agent_velocities):
        input_data = torch.tensor(agent_positions + agent_velocities, dtype=torch.float32).unsqueeze(0).to(device)
        predicted_trajectories = self.forward(input_data)
        return predicted_trajectories.squeeze(0).detach().cpu().numpy()

In [3]:
#Environment
class ContinuousRobotNavigationEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    def is_goal_behind_wall(self, goal_position):
        # Check if goal_position is behind any wall
        for wall in self.maze_walls:
            x, y, width, height = wall
            if goal_position[0] >= x and goal_position[0] <= x + width \
                    and goal_position[1] >= y and goal_position[1] <= y + height:
                return True
        return False
    def __init__(self, interaction_model):
        super(ContinuousRobotNavigationEnv, self).__init__()
        self.grid_size = 10
        self.observation_space = spaces.Box(low=0, high=self.grid_size, shape=(2,), dtype=np.float32)
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)  # Continuous action space

        self.velocity_scale = 0.3
        self.lidar_range = 3.0

        self.num_humans = 5  # Number of humans
        self.human_positions = [self._random_position() for _ in range(self.num_humans)]
        self.human_velocities = [self._random_velocity() for _ in range(self.num_humans)]
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]
        
        # self.num_obstacles = 3  # Number of static obstacles
        # self.obstacle_positions = [self._random_position() for _ in range(self.num_obstacles)]

        self.maze_walls = [
            (5, 8, 0.3, 1.5),   # Example wall 1
            (0, 7, 1.5, 0.3),   # Example wall 2
            (6.5, 6.5, 1.5, 0.3),   # Example wall 3
            (4, 4.5, 0.3, 1.5),   # Example wall 4
            (1.5, 2, 0.3, 1.5), #5th wall
            (5, 2.5, 1.5, 0.3),  #6th wall
            (8, 4, 1.5, 0.3),    #7th wall
            (7, 0, 0.3, 1.5)   #8th wall    
        ]

        
        
        self.goal_position = (0, 0)

        self.frames = []  # Initialize the frames list
        self.interaction_model = interaction_model

        
        self.reset()

    def _random_position(self):
        return [np.random.uniform(0, self.grid_size - 1), np.random.uniform(0, self.grid_size - 1)]

    def _random_velocity(self):
        # return [np.random.uniform(-1, 1), np.random.uniform(-1, 1)]
        return [np.random.uniform(-1, 1) * self.velocity_scale, np.random.uniform(-1, 1) * self.velocity_scale]

    def reset(self):
        self.robot_position = [0, 0]
        self.robot_orientation = np.pi/4  #looking diagonally up
        self.human_positions = [self._random_position() for _ in range(self.num_humans)]
        self.human_velocities = [self._random_velocity() for _ in range(self.num_humans)]
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]
        # self.obstacle_positions = [self._random_position() for _ in range(self.num_obstacles)]
        self.state = np.array(self.robot_position, dtype=np.float32)
        # self.goal_position = self._random_position()
        while True:
            self.goal_position = self._random_position()
            if not self.is_goal_behind_wall(self.goal_position):
                break
        self.frames = []  # Reset frames list at the start of each episode
        
        return self.state

    def check_wall_collision(self, position):
        for wall in self.maze_walls:
            x, y, width, height = wall
            if position[0] >= (x-0.1) and position[0] <= x + width and position[1] >= (y-0.1) and position[1] <= y + height:
                return True
        return False

    def prepare_trajectory_data(self):
        agent_positions = self.human_positions + [self.robot_position]
        agent_velocities = self.human_velocities + [[0, 0]]  # Robot velocity can be assumed as [0, 0] initially
        return agent_positions, agent_velocities
        
    def predict_human_positions(self):
        self.predicted_human_positions = [[] for _ in range(self.num_humans)]  # Reset predicted positions
        num_predictions = 5  # Number of future positions to predict
        
        for i in range(self.num_humans):
            future_positions = []
            future_position = np.array(self.human_positions[i])
            for _ in range(num_predictions):
                future_position += np.array(self.human_velocities[i])
                future_position[0] = np.clip(future_position[0], 0, self.grid_size - 1)
                future_position[1] = np.clip(future_position[1], 0, self.grid_size - 1)
                future_positions.append(future_position.copy())
        
            # Update current position
            self.human_positions[i][0] += self.human_velocities[i][0]
            self.human_positions[i][1] += self.human_velocities[i][1]
        
            # Check for boundary collisions and reverse velocity if necessary
            if self.human_positions[i][0] < 0 or self.human_positions[i][0] >= self.grid_size:
                self.human_velocities[i][0] *= -1
            if self.human_positions[i][1] < 0 or self.human_positions[i][1] >= self.grid_size:
                self.human_velocities[i][1] *= -1

            if self.check_wall_collision(self.human_positions[i]):
                self.human_velocities[i][0] *= -1
                self.human_velocities[i][1] *= -1
        
            # Update future positions if within lidar range
            if np.linalg.norm(np.array(self.robot_position) - np.array(self.human_positions[i])) <= self.lidar_range:
                self.predicted_human_positions[i] = future_positions
        


    def check_collision(self, position):
        # Check collision with humans
        for human_pos in self.human_positions:
            if np.linalg.norm(np.array(self.robot_position) - np.array(human_pos)) < 1.0:  # Collision threshold
                return True
        
        # Check collision with obstacles
        # for obstacle_pos in self.obstacle_positions:
        #     if np.linalg.norm(np.array(self.robot_position) - np.array(obstacle_pos)) < 1.0:  # Collision threshold
        #         return True

        for wall in self.maze_walls:
            x, y, width, height = wall
            if position[0] >= (x-0.3) and position[0] <= x + width and position[1] >= (y-0.3) and position[1] <= y + height:
                return True
        
        return False

        
    def step(self, action):
        self.state = np.array(self.robot_position, dtype=np.float32)
        # Apply scaling to the action to reduce velocity
        scaled_action = np.array(action) * self.velocity_scale
    
        # Calculate the new robot position
        new_robot_position = [
            self.robot_position[0] + scaled_action[0],
            self.robot_position[1] + scaled_action[1]
        ]
    
        # Clip the new position to stay within the environment bounds
        new_robot_position = [
            np.clip(new_robot_position[0], 0, self.grid_size - 1),
            np.clip(new_robot_position[1], 0, self.grid_size - 1)
        ]
    
        # Check if the new position collides with maze walls
        if not self.check_collision(new_robot_position):
            self.robot_position = new_robot_position
        else:
            done = True
            return self.state, -10, done, {}
    
        if np.linalg.norm(scaled_action) > 0:
            self.robot_orientation = np.arctan2(scaled_action[1], scaled_action[0])

        agent_positions, agent_velocities = self.prepare_trajectory_data()
        predicted_trajectories = self.interaction_model.predict(agent_positions, agent_velocities)
        # print(predicted_trajectories)
        self.predict_human_positions()
        
        goal_threshold = 0.7
        distance_to_goal = np.linalg.norm(np.array(self.robot_position) - np.array(self.goal_position))

        reward = -distance_to_goal

        done = False

        if distance_to_goal < 0.7: 
            reward += 100  
            done = True
        else:
            collision = self.check_collision(self.robot_position)
            if collision:
                reward -= 10  
                done = True
            else:
                done = False
    
        # if done:
        #     self.reset()  
        
        self.state = np.array(self.robot_position, dtype=np.float32)
        # print(self.state)
        info = {}  # Add any additional info you want to pass
    
        return self.state, reward, done, info
        
    def render(self, mode='human'):
        plt.figure(figsize=(6, 6))
        ax = plt.gca()
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_aspect('equal')
    
        # Draw the goal position
        goal = plt.Circle(self.goal_position, 0.5, color='green')
        ax.add_artist(goal)
    
        # Draw the robot
        robot = plt.Circle(self.robot_position, 0.5, color='red')
        ax.add_artist(robot)
    
        # Draw the humans
        for human_pos in self.human_positions:
            human = plt.Circle(human_pos, 0.5, color='blue')
            ax.add_artist(human)

        
        #static obstacles
        # for obstacle_pos in self.obstacle_positions:
        #     obstacle = plt.Circle(obstacle_pos, 0.5, color='black')
        #     ax.add_artist(obstacle)

        #walls
        for wall in self.maze_walls:
            x, y, width, height = wall
            rect = plt.Rectangle((x, y), width, height, color='black')
            ax.add_patch(rect)
    
        
        #Orientation
        arrow_length = 0.7
        dx = arrow_length * np.cos(self.robot_orientation)
        dy = arrow_length * np.sin(self.robot_orientation)
        arrow = FancyArrow(self.robot_position[0], self.robot_position[1], dx, dy, color='black', width=0.1)
        ax.add_patch(arrow)

        #Lidar range
        # lidar_circle = plt.Circle(self.robot_position, self.lidar_range, color='gray', alpha=0.2)
        # ax.add_artist(lidar_circle)
        self.render_lidar(ax)

        #Trajectory prediction
        for predicted_positions in self.predicted_human_positions:
            for future_pos in predicted_positions:
                future_circle = plt.Circle(future_pos, 0.2, color='pink', alpha=0.3)
                ax.add_artist(future_circle)

        
    
        plt.axis('off')
    
        # Save frame to list
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        frame = imageio.imread(buf)
        self.frames.append(frame)
        buf.close()

        plt.close()  

        # Show the plot in real-time
        # plt.imshow(frame)
        # plt.axis('off')
        # plt.pause(0.1)
        # plt.clf()

    def render_lidar(self, ax):
        lidar_segments = 36
        angle_increment = 2 * np.pi / lidar_segments
        for i in range(lidar_segments):
            angle = i * angle_increment
            end_point = self.robot_position + np.array([np.cos(angle), np.sin(angle)]) * self.lidar_range
            ray_end_point = self.get_ray_end_point(self.robot_position, end_point)
            ax.plot([self.robot_position[0], ray_end_point[0]], [self.robot_position[1], ray_end_point[1]], color='dimgray', alpha=0.2)

    def get_ray_end_point(self, start, end):
        for wall in self.maze_walls:
            x, y, width, height = wall
            if self.line_intersects_rect(start, end, x, y, width, height):
                return self.get_intersection_point(start, end, x, y, width, height)
        return end

    def line_intersects_rect(self, start, end, x, y, width, height):
        def ccw(A, B, C):
            return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

        A = start
        B = end
        C1 = [x, y]
        C2 = [x + width, y]
        C3 = [x, y + height]
        C4 = [x + width, y + height]
        return (ccw(A, C1, C2) != ccw(B, C1, C2) and ccw(A, B, C1) != ccw(A, B, C2)) or \
               (ccw(A, C3, C4) != ccw(B, C3, C4) and ccw(A, B, C3) != ccw(A, B, C4)) or \
               (ccw(A, C1, C3) != ccw(B, C1, C3) and ccw(A, B, C1) != ccw(A, B, C3)) or \
               (ccw(A, C2, C4) != ccw(B, C2, C4) and ccw(A, B, C2) != ccw(A, B, C4))

    def get_intersection_point(self, start, end, x, y, width, height):
        def line(p1, p2):
            A = (p1[1] - p2[1])
            B = (p2[0] - p1[0])
            C = (p1[0] * p2[1] - p2[0] * p1[1])
            return A, B, -C

        def intersection(L1, L2):
            D = L1[0] * L2[1] - L1[1] * L2[0]
            Dx = L1[2] * L2[1] - L1[1] * L2[2]
            Dy = L1[0] * L2[2] - L1[2] * L2[0]
            if D != 0:
                return Dx / D, Dy / D
            else:
                return False

        walls = [
            ([x, y], [x + width, y]),
            ([x, y], [x, y + height]),
            ([x + width, y], [x + width, y + height]),
            ([x, y + height], [x + width, y + height])
        ]
        closest_point = end
        min_distance = np.linalg.norm(np.array(start) - np.array(end))
        for wall in walls:
            L1 = line(start, end)
            L2 = line(wall[0], wall[1])
            R = intersection(L1, L2)
            if R:
                dist = np.linalg.norm(np.array(start) - np.array(R))
                if dist < min_distance:
                    min_distance = dist
                    closest_point = R
        return closest_point
    

In [4]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, action_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.tanh(self.fc3(x))  # Output action is in range [-1, 1]
        return x

class Critic(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_size + action_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)
    
    def forward(self, state, action):
        x = torch.cat((state, action), dim=-1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
class OUNoise:
    def __init__(self, action_dim, mu=0.0, theta=0.15, sigma=0.2):
        self.action_dim = action_dim
        self.mu = mu
        self.theta = theta
        self.sigma = sigma
        self.reset()

    def reset(self):
        self.state = np.ones(self.action_dim) * self.mu

    def sample(self):
        dx = self.theta * (self.mu - self.state) + self.sigma * np.random.randn(self.action_dim)
        self.state += dx
        return self.state

In [17]:
class DDPGAgent:
    def __init__(self, state_size, action_size, buffer_size, batch_size, lr_actor=1e-3, lr_critic=1e-3, gamma=0.99, tau=1e-3, huber_delta=1.0, target_update_freq=10, noise_std=0.05):
        self.actor = Actor(state_size, action_size)
        self.actor_target = Actor(state_size, action_size)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_actor)
        
        self.critic = Critic(state_size, action_size)
        self.critic_target = Critic(state_size, action_size)
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_critic)
        
        self.replay_buffer = ReplayBuffer(buffer_size, batch_size, state_size, action_size)
        self.batch_size = batch_size
        self.gamma = gamma
        self.tau = tau
        self.huber_delta = huber_delta
        self.target_update_freq = target_update_freq
        self.step_counter = 0
        self.noise = OUNoise(action_size, sigma=noise_std)
        
        # Initialize target networks with same weights as original networks
        self._update_targets(self.actor_target, self.actor)
        self._update_targets(self.critic_target, self.critic)

        #losses
        self.actor_losses = []
        self.critic_losses = []
    
    def _update_targets(self, target_model, model):
        for target_param, param in zip(target_model.parameters(), model.parameters()):
            target_param.data.copy_(param.data)
            
    def act(self, state, add_noise=True):
        state = torch.FloatTensor(state)
        action = self.actor(state).detach().numpy().flatten()
        if add_noise:
            action += self.noise.sample()  # Add noise
        return np.clip(action, -1, 1)  # Clip action to be within valid range
        # return self.actor(state).detach().numpy()
        
    #Test
    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.add(state, action, reward, next_state, done)
        
    #Learn
    def learn(self):
        if len(self.replay_buffer) < self.batch_size:
            return
        
        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        states = torch.FloatTensor(states)
        actions = torch.FloatTensor(actions)
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones).unsqueeze(1)
        
        # Critic update
        Q_targets_next = self.critic_target(next_states, self.actor_target(next_states))
        Q_targets = rewards + (self.gamma * Q_targets_next * (1 - dones))
        Q_expected = self.critic(states, actions)
        # critic_loss = F.mse_loss(Q_expected, Q_targets.detach())
        critic_loss = F.smooth_l1_loss(Q_expected, Q_targets.detach()) #huber loss
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic.parameters(), 1.0)  # Gradient clipping
        self.critic_optimizer.step()
        
        # Actor update
        actor_loss = -self.critic(states, self.actor(states)).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 1.0)  # Gradient clipping
        self.actor_optimizer.step()

        self.actor_losses.append(actor_loss.item())
        self.critic_losses.append(critic_loss.item())
        
        #Update target networks based on step counter
        self.step_counter += 1
        if self.step_counter % self.target_update_freq == 0:
            self._soft_update_target(self.actor_target, self.actor, self.tau)
            self._soft_update_target(self.critic_target, self.critic, self.tau)


    def plot_loss(self, save_path=None):
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.plot(self.actor_losses, label='Actor Loss')
        plt.xlabel('Training Steps')
        plt.ylabel('Loss')
        plt.title('Actor Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(self.critic_losses, label='Critic Loss')
        plt.xlabel('Training Steps')
        plt.ylabel('Loss')
        plt.title('Critic Loss')
        plt.legend()

        plt.tight_layout()
        plt.show()
    
    def _soft_update_target(self, target_model, model, tau):
        for target_param, param in zip(target_model.parameters(), model.parameters()):
            target_param.data.copy_(tau * param.data + (1.0 - tau) * target_param.data)


In [6]:
class ReplayBuffer:
    def __init__(self, max_size, batch_size, state_size, action_size):
        self.max_size = max_size
        self.batch_size = batch_size
        self.state_size = state_size
        self.action_size = action_size
        self.buffer = []
        self.idx = 0
        
    def add(self, state, action, reward, next_state, done):
        experience = (state, action, reward, next_state, done)
        if len(self.buffer) < self.max_size:
            self.buffer.append(experience)
        else:
            self.buffer[self.idx] = experience
        self.idx = (self.idx + 1) % self.max_size
        
    def sample(self, batch_size):
        batch = np.random.choice(len(self.buffer), batch_size, replace=False)
        states, actions, rewards, next_states, dones = zip(*[self.buffer[i] for i in batch])
        return states, actions, rewards, next_states, dones
    
    def __len__(self):
        return len(self.buffer)

# class PrioritizedReplayBuffer:
#     def __init__(self, capacity, alpha=0.6):
#         self.capacity = capacity
#         self.alpha = alpha
#         self.buffer = []
#         self.priorities = []
#         self.position = 0

#     def add(self, state, action, next_state, reward, done):
#         max_priority = max(self.priorities, default=1.0)
#         experience = (state, action, next_state, reward, done)
#         if len(self.buffer) < self.capacity:
#             self.buffer.append(experience)
#             self.priorities.append(max_priority)
#         else:
#             self.buffer[self.position] = experience
#             self.priorities[self.position] = max_priority
#         self.position = (self.position + 1) % self.capacity

#     def sample(self, batch_size, beta=0.4):
#         priorities = np.array(self.priorities)
#         probabilities = priorities ** self.alpha
#         probabilities /= probabilities.sum()

#         indices = np.random.choice(len(self.buffer), batch_size, p=probabilities)
#         samples = [self.buffer[idx] for idx in indices]

#         total = len(self.buffer)
#         weights = (total * probabilities[indices]) ** (-beta)
#         weights /= weights.max()

#         batch = list(zip(*samples))
#         states = np.array(batch[0])
#         actions = np.array(batch[1])
#         next_states = np.array(batch[2])
#         rewards = np.array(batch[3])
#         dones = np.array(batch[4])

#         return states, actions, next_states, rewards, dones, weights, indices

#     def update_priorities(self, batch_indices, batch_priorities):
#         for idx, priority in zip(batch_indices, batch_priorities):
#             self.priorities[idx] = priority

In [7]:
def save_checkpoint(actor, critic, actor_target, critic_target, actor_optimizer, critic_optimizer, filepath):
    checkpoint = {
        'actor_state_dict': actor.state_dict(),
        'critic_state_dict': critic.state_dict(),
        'actor_target_state_dict': actor_target.state_dict(),
        'critic_target_state_dict': critic_target.state_dict(),
        'actor_optimizer_state_dict': actor_optimizer.state_dict(),
        'critic_optimizer_state_dict': critic_optimizer.state_dict()
    }
    torch.save(checkpoint, filepath)
    print(f"Checkpoint saved to {filepath}")

def load_checkpoint(filepath, actor, critic, actor_target, critic_target, actor_optimizer, critic_optimizer):
    checkpoint = torch.load(filepath)
    actor.load_state_dict(checkpoint['actor_state_dict'])
    critic.load_state_dict(checkpoint['critic_state_dict'])
    actor_target.load_state_dict(checkpoint['actor_target_state_dict'])
    critic_target.load_state_dict(checkpoint['critic_target_state_dict'])
    actor_optimizer.load_state_dict(checkpoint['actor_optimizer_state_dict'])
    critic_optimizer.load_state_dict(checkpoint['critic_optimizer_state_dict'])
    print(f"Checkpoint loaded from {filepath}")
    

In [21]:
def save_loss_data(filename, actor_losses, critic_losses):
    loss_data = {
        "actor_losses": actor_losses,
        "critic_losses": critic_losses
    }
    with open(filename, 'w') as f:
        json.dump(loss_data, f)

def load_loss_data(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            loss_data = json.load(f)
        return loss_data["actor_losses"], loss_data["critic_losses"]
    else:
        return [], []

def save_episode_count(filename, episode_count):
    with open(filename, 'w') as f:
        json.dump({'episode_count': episode_count}, f)

def load_episode_count(filename):
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            data = json.load(f)
            return data.get('episode_count', 0)
    return 0

In [23]:
def main():
    interaction_model = InteractionBasedAttentionModel(input_dim=2, hidden_dim=128, output_dim=2, num_heads=4, num_layers=6)
    interaction_model.to(device)
    env = ContinuousRobotNavigationEnv(interaction_model)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    buffer_size = 10000
    batch_size = 64
    lr_actor = 1e-3
    lr_critic = 1e-3
    gamma = 0.99
    tau = 1e-3

    ddpg_agent = DDPGAgent(state_dim, action_dim, buffer_size, batch_size, lr_actor, lr_critic, gamma, tau)
    replay_buffer = ReplayBuffer(max_size=buffer_size, batch_size=batch_size, state_size=state_dim, action_size=action_dim)
    # replay_buffer = PrioritizedReplayBuffer(capacity=100000)

    checkpoint_dir = 'checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    # Define checkpoint filename with timestamp
    checkpoint_filename = f"ddpg_checkpoint_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pth"
    checkpoint_filepath = os.path.join(checkpoint_dir, checkpoint_filename)

    # if os.path.exists(checkpoint_filepath):
    #     load_checkpoint(checkpoint_filepath, ddpg_agent.actor, ddpg_agent.critic, ddpg_agent.actor_target,
    #                     ddpg_agent.critic_target, ddpg_agent.actor_optimizer, ddpg_agent.critic_optimizer)
    
    checkpoint_files = sorted([f for f in os.listdir(checkpoint_dir) if f.endswith('.pth')], reverse=True)
    latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_files[0]) if checkpoint_files else None

    if latest_checkpoint:
        print(f"Loading checkpoint {latest_checkpoint}")
        load_checkpoint(latest_checkpoint, ddpg_agent.actor, ddpg_agent.critic, ddpg_agent.actor_target,
                        ddpg_agent.critic_target, ddpg_agent.actor_optimizer, ddpg_agent.critic_optimizer)

    loss_data_filepath = os.path.join(checkpoint_dir, 'loss_data.json')
    ddpg_agent.actor_losses, ddpg_agent.critic_losses = load_loss_data(loss_data_filepath)

    episode_count_filepath = os.path.join(checkpoint_dir, 'episode_count.json')
    episode_count = load_episode_count(episode_count_filepath)

    num_episodes_per_run = 200
    total_episodes = 1000
    num_runs = (total_episodes - episode_count) // num_episodes_per_run

    print(f"Starting training run with episode{num_runs+1}...")

    # Training loop
    episode_rewards = []

    for episode in range(num_episodes_per_run):
        state = env.reset()
        episode_reward = 0
        env.frames = []  # Clear frames for the new episode
        for t in range(1000):
            action = ddpg_agent.act(state)
            agent_positions, agent_velocities = env.prepare_trajectory_data()
            predicted_trajectories = interaction_model.predict(agent_positions, agent_velocities)
            # print(predicted_trajectories)
            next_state, reward, done, _ = env.step(action)
            ddpg_agent.remember(state, action, reward, next_state, done)  # Remember the experience
            state = next_state
            episode_reward += reward
            env.render()  
            if done:
                break

        if (episode+1) % 100 == 0:  # Save checkpoint every 100 episodes
            save_checkpoint(ddpg_agent.actor, ddpg_agent.critic, ddpg_agent.actor_target, 
                            ddpg_agent.critic_target, ddpg_agent.actor_optimizer, ddpg_agent.critic_optimizer, checkpoint_filepath)
            print(f'Saved checkpoint at episode {episode}')
        
        # Learn from the experiences in the replay buffer
        ddpg_agent.learn()

        episode_rewards.append(episode_reward)
        print(f"Episode {episode}, Reward: {episode_reward}")

        # Save GIF
        imageio.mimsave(f'episode_{episode}.gif', env.frames, fps=10)

        save_loss_data(loss_data_filepath, ddpg_agent.actor_losses, ddpg_agent.critic_losses)
    episode_count += num_episodes_per_run
    save_episode_count(episode_count_filepath, episode_count)
        

    plt.plot(range(num_episodes_per_run), episode_rewards)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward')
    plt.title('Cumulative Reward over Episodes')
    plt.savefig('cumulative_reward_plot.png')
    plt.show()

    ddpg_agent.plot_loss(save_path=f'loss_plot_episode_{episode}.png')
    ddpg_agent.plot_loss()



if __name__ == "__main__":
    main()

Loading checkpoint checkpoints/ddpg_checkpoint_20240719_145402.pth
Checkpoint loaded from checkpoints/ddpg_checkpoint_20240719_145402.pth
Starting training run with episode6...
Checkpoint saved to checkpoints/ddpg_checkpoint_20240719_150950.pth
Saved checkpoint at episode 0
Episode 0, Reward: -59.7806951854767
Episode 1, Reward: -41.520631964935134
Episode 2, Reward: -51.17748245913477
Episode 3, Reward: -54.06735628633688
Episode 4, Reward: -148.7317058115546
Episode 5, Reward: -19.798425351402166
Episode 6, Reward: -61.00698090497154
Episode 7, Reward: -41.415165427372365
Episode 8, Reward: -44.0086025153768
Episode 9, Reward: -40.004746195290004
Episode 10, Reward: -3.2886076191947637
Episode 11, Reward: -98.74965639056298
Episode 12, Reward: -84.86046708702845
Episode 13, Reward: -52.12929577412136
Episode 14, Reward: -54.1717556585542
Episode 15, Reward: -10
Episode 16, Reward: -71.20358201920556
Episode 17, Reward: -27.391189843007396
Episode 18, Reward: 99.92603642757052
Episode

Exception ignored in: <function PluginV3.__del__ at 0x12a538ca0>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/imageio/core/v3_plugin_api.py", line 370, in __del__
    self.close()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/imageio/plugins/pillow.py", line 144, in close
    self._flush_writer()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/imageio/plugins/pillow.py", line 485, in _flush_writer
    primary_image.save(self._request.get_file(), **self.save_args)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/PIL/Image.py", line 2413, in save
    save_handler(self, fp, filename)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/PIL/GifImagePlugin.py", line 653, in _save_all
    _save(im, fp, filename, save_all=True)
  File "/Library/Frameworks/Python.

Episode 31, Reward: -200.9606327118118
Episode 32, Reward: -120.56655456088961
Episode 33, Reward: -10
Episode 34, Reward: -55.89581462314272
Episode 35, Reward: -156.49149198203767
Episode 36, Reward: -51.243586127336144
Episode 37, Reward: -85.93620336247076
Episode 38, Reward: 90.58502842794839
Episode 39, Reward: -246.6795810016423
Episode 40, Reward: -89.14223371152855
Episode 41, Reward: -141.09372300405036
Episode 42, Reward: -87.36593473946841
Episode 43, Reward: -226.12500118810365
Episode 44, Reward: -33.658254105763675
Episode 45, Reward: -276.8697395567752
Episode 46, Reward: -238.049174064195
Episode 47, Reward: -17.518302287663587
Episode 48, Reward: -251.68773148743244
Episode 49, Reward: -105.5028993883232
Episode 50, Reward: -142.25669499864304
Episode 51, Reward: -36.47859575498454
Episode 52, Reward: -284.35249795875484
Episode 53, Reward: -226.71861887144254
Episode 54, Reward: -235.7406672420259
Episode 55, Reward: -196.89568004137362
Episode 56, Reward: -215.14193

NameError: name 'num_episodes' is not defined